## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details. You can download the required DB from the shared dropbox or from blackboard

In [104]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from string import punctuation
import string
from nltk.corpus import stopwords
# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

In [159]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [160]:
# Define the table name
conv = 'conventions'

# Fetch all rows from the table
query = f"SELECT * FROM {conv};"
df = pd.read_sql_query(query, conv_conn)
print (df.shape)
df.head()

(2541, 8)


,party,night,speaker,speaker_count,time,text,text_len,file
0,Democratic,4,Unknown,1,00:00,Skip to content The Company Careers Press Free...,127,www_rev_com_blog_transcripts2020-democratic-na...
1,Democratic,4,Speaker 1,1,00:33,I’m here by calling the full session of the 48...,41,www_rev_com_blog_transcripts2020-democratic-na...
2,Democratic,4,Speaker 2,1,00:59,"Every four years, we come together to reaffirm...",17,www_rev_com_blog_transcripts2020-democratic-na...
3,Democratic,4,Kerry Washington,1,01:07,We fight for a more perfect union because we a...,28,www_rev_com_blog_transcripts2020-democratic-na...
4,Democratic,4,Bernie Sanders,1,01:18,"We must come together to defeat Donald Trump, ...",22,www_rev_com_blog_transcripts2020-democratic-na...


#### Clean and tokenize text function

In [161]:
punctuation = set(string.punctuation)

# Function to clean and tokenize the lyrics
def clean_and_tokenize(text):
    if text is None:
        return []
    # Remove punctuation
    clean_text = ''.join([c for c in text if c not in punctuation])
    # Convert tokens to lowercase
    clean_text = clean_text.lower()
    # Tokenize the text
    tokens = word_tokenize(clean_text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    #Join tokens into a single string
    string_tokens = ' '.join(tokens)
    return string_tokens
  

In [162]:
# Execute the query to retrieve the "text" column from the "convention" table
query = '''
    SELECT text, party
    FROM conventions
'''

convention_cur.execute(query)

# Fetch all the results
results = convention_cur.fetchall()

# Populate convention_data with the cleaned and tokenized text and party information
convention_data = []
for row in results:
    text = row[0]
    party = row[1]

    # Clean and tokenize the text
    cleaned_text = clean_and_tokenize(text)

    # Append the cleaned and tokenized text and party information as a sublist to convention_data
    convention_data.append([cleaned_text, party])


Let's look at some random entries and see if they look right. 

In [163]:
random.choices(convention_data,k=2)

[['seems like yesterday first convention husband accepted republican nomination became 45th president united states yet energy enthusiasm lead nation real today four years ago know speak husband entire family say forgotten incredible people willing take chance businessmen never worked politics know elected commanderinchief know carry us humbled incredible support still grateful today',
  'Republican'],
 ['takes trust get things done divided government think joe biden colleagues knew points equally valid',
  'Democratic']]

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least word_cutoff times. Here's the code to test that if you want it.

In [123]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2332 as features in the model.


In [125]:
def conv_features(text, fw):
    """Given some text, this returns a dictionary holding the feature words.
    
    Args:
        text (str): A piece of text in a continuous string. Assumes text has been cleaned and case-folded.
        fw (set): The feature words that we're considering. A word in `text` must be in `fw` in order to be returned.
                  This prevents us from considering very rarely occurring words.
        
    Returns:
        dict: A dictionary with the words in `text` that appear in `fw`. Words are only counted once.
              If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'}, then this would return
              a dictionary of {'quick': True, 'fox': True}.
    """
    ret_dict = {}
    
    for word in text.split():
        if word in fw:
            ret_dict[word] = True
          
    return ret_dict


In [126]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald': True,'president': True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory.

In [131]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [132]:
random.seed(509)
random.shuffle(featuresets)

test_size = 500

In [133]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.522


In [134]:
classifier.show_most_informative_features(25)

Most Informative Features
                 radical = True           Republ : Democr =     37.0 : 1.0
                   votes = True           Democr : Republ =     24.7 : 1.0
               greatness = True           Republ : Democr =     22.0 : 1.0
                 destroy = True           Republ : Democr =     19.8 : 1.0
             enforcement = True           Republ : Democr =     19.6 : 1.0
                freedoms = True           Republ : Democr =     16.6 : 1.0
                   china = True           Republ : Democr =     15.2 : 1.0
                   taxes = True           Republ : Democr =     15.1 : 1.0
                   media = True           Republ : Democr =     14.9 : 1.0
               countries = True           Republ : Democr =     14.5 : 1.0
                  defund = True           Republ : Democr =     14.5 : 1.0
                 beliefs = True           Republ : Democr =     13.4 : 1.0
                    mike = True           Republ : Democr =     12.6 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

The Naive Bayes classifier shows a relatively low accuracy, approximately 52.2%, implying that it correctly predicts the party affiliation for only about 52.2% of the instances in the test set.

The output from the 25 most informative features of the classifier provides valuable insights. It shows that many of these features have higher likelihood ratios for the Republican party compared to the Democratic party. Notably, specific words or topics demonstrate a stronger association with the Republican party in the analyzed text data, including terms like "radical," "greatness," "destroy," "enforcement," and "freedoms." These words indicate their prominence in Republican-oriented language and suggest their significance in capturing the distinctive characteristics of Republican text.

## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [165]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [166]:
# Execute the query to retrieve table names
cong_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all the table names
tables = cong_cur.fetchall()

# Print the table names
for table in tables:
    print(table[0])


websites
candidate_data
tweets


In [167]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [174]:
results[:3]

[('Mo Brooks',
  'Republican',
  b'"Brooks Joins Alabama Delegation in Voting Against Flawed Funding Bill" http://t.co/3CwjIWYsNq'),
 ('Mo Brooks',
  'Republican',
  b'"Brooks: Senate Democrats Allowing President to Give Americans\xe2\x80\x99 Jobs to Illegals" #securetheborder https://t.co/mZtEaX8xS6'),
 ('Mo Brooks',
  'Republican',
  b'"NASA on the Square" event this Sat. 11AM \xe2\x80\x93 4PM. Stop by &amp; hear about the incredible work done in #AL05! @DowntownHSV http://t.co/R9zY8WMEpA')]

In [169]:
# Populate tweet_data with the cleaned and tokenized text and party information
tweet_data = []
for row in results:
    text = row[2].decode('utf-8') # Decode the byte string using UTF-8
    party = row[1]

    # Clean and tokenize the text
    cleaned_text = clean_and_tokenize(text)

    # Append the cleaned and tokenized text and party information as a sublist to convention_data
    tweet_data.append([cleaned_text, party])


In [176]:
print (tweet_data[:3])

[['brooks joins alabama delegation voting flawed funding bill httptco3cwjiwysnq', 'Republican'], ['brooks senate democrats allowing president give americans ’ jobs illegals securetheborder httpstcomzteax8xs6', 'Republican'], ['nasa square event sat 11am – 4pm stop amp hear incredible work done al05 downtownhsv httptcor9zy8wmepa', 'Republican']]


There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [183]:
random.seed(509)
tweet_data_sample = random.choices(tweet_data,k=10)

In [186]:
for tweet, party in tweet_data_sample:
    featuresets = conv_features(tweet, feature_words)
    estimated_party = classifier.classify(featuresets)
    # Estimate the actual party using the classifier
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifier says {estimated_party}.")
    print("")

Here's our (cleaned) tweet: sent video honor veterans republican womens club katy httpstcoukopee1l7c txlege
Actual party is Republican and our classifier says Republican.

Here's our (cleaned) tweet: continue visits working families already underway austin san antonio places
Actual party is Democratic and our classifier says Republican.

Here's our (cleaned) tweet: blueeyes92467 rosie ’ get judgment woman made mistake apologized ’ kill anyone
Actual party is Republican and our classifier says Republican.

Here's our (cleaned) tweet: billkellytexas according close friend mine god hates patriots
Actual party is Democratic and our classifier says Republican.

Here's our (cleaned) tweet: proverbs 19 dont neglect mothers teaching learn crown grace clothe honor
Actual party is Republican and our classifier says Republican.



Now that we've looked at it some, let's score a bunch and see how we're doing.

In [193]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    featuresets = conv_features(tweet, feature_words)
    estimated_party = classifier.classify(featuresets)
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [194]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 3589, 'Democratic': 679}),
             'Democratic': defaultdict(int,
                         {'Republican': 4663, 'Democratic': 1071})})

### Reflections

_Write a little about what you see in the results_ 

Answer:the result shows that the  classifier has  a higher accuracy in predicting tweets labeled as 'Republican'. It correctly classified 3589 out of 4268 'Republican' tweets, and misclassified 679 'Republican' tweets as 'Democratic'. On the other hand, for tweets labeled as 'Democratic', the classifier give a lower accuracy. It correctly classified 1017 out of 5734 as 'Democratic', and misclassified 4663 as 'Republican' . This also aligns with observation earlier when we look at the most important features of the classifier which seems to be more sensitive to certain features associated with the 'Republican' party.